In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from transformers import LogTransformer, SquareRootTransformer, ZScoreTrimmer, UpperBoundTrimmer, LowerBoundTrimmer, FillNaColumnTransformer, KNNColumnImputer, OneHotColumnEncoder, OrdinalColumnEncoder, ColumnDropper, StandardColumnScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

In [3]:
df = pd.read_csv("data/belgian_property_prices.csv")

In [ ]:
features_to_drop = ["external_reference", "latest_land_use_designation", "website", "reference_number_of_the_epc_report", "street", "housenumber", "city", "postal", "state"]

In [ ]:
numerical_features = [
    "bathrooms",
    "bedroom_1_surface",
    "bedroom_2_surface",
    "bedroom_3_surface",
    "bedrooms",
    "co2_emission",
    "cadastral_income",
    "construction_year",
    "covered_parking_spaces",
    "garden_surface",
    "kitchen_surface",
    "living_area",
    "living_room_surface",
    "outdoor_parking_spaces",
    "primary_energy_consumption",
    "street_frontage_width",
    "surface_of_the_plot",
    "toilets",
    "width_of_the_lot_on_the_street",
    "yearly_theoretical_total_energy_consumption",
    "lat",
    "lng"
    ]

In [ ]:
categorical_features = [
    "as_built_plan",
    "basement",
    "connection_to_sewer_network",
    "dining_room",
    "double_glazing",
    "flood_zone_type",
    "furnished",
    "gas_water__electricity",
    "office",
    "planning_permission_obtained",
    "possible_priority_purchase_right",
    "proceedings_for_breach_of_planning_regulations",
    "subdivision_permit",
    "surroundings_type",
    "tv_cable",
    "tenement_building",
    "available_as_of",
    "building_condition",
    "number_of_frontages",
    "energy_class",
    "planning_permission_obtained", 
    "heating_type"
    ]

In [ ]:
df["price_cat"] = pd.cut(df.price, bins=[0,250000,500000,750000,1000000,np.inf], labels=[0,1,2,3,4])
train_set, test_set = train_test_split(df, test_size=0.2, stratify=df.price_cat, random_state=42)

for set_ in (train_set, test_set):
    set_.drop("price_cat", axis=1, inplace=True)

In [ ]:
pipeline = Pipeline([
    ("column_dropping", ColumnDropper(columns=features_to_drop)),
    ("log_transforms", LogTransformer(columns=[
        "co2_emission",
        "cadastral_income",
        "garden_surface",
        "kitchen_surface",
        "living_area",
        "living_room_surface",
        "street_frontage_width",
        "surface_of_the_plot",
        "width_of_the_lot_on_the_street", 
        "price"
    ])),
    ("square_root_transforms", SquareRootTransformer(columns=[
        "bathrooms",
        "bedroom_1_surface",
        "primary_energy_consumption",
        "yearly_theoretical_total_energy_consumption"
    ])),
    ("zscore_trimming", ZScoreTrimmer(columns=[
        "living_room_surface",
        "primary_energy_consumption",
        "yearly_theoretical_total_energy_consumption"
    ])),                                           
    ("upperbound_trimming", UpperBoundTrimmer(column_boundaries={
        "bathrooms": 3,
        "bedroom_1_surface":8,
        "bedroom_2_surface":40,
        "bedroom_3_surface":40,
        "bedrooms":10,
        "co2_emission": 10,
        "covered_parking_spaces": 10,
        "garden_surface":11,
        "kitchen_surface":4,
        "outdoor_parking_spaces":20,
        "street_frontage_width": 4,
        "toilets":10,
        "width_of_the_lot_on_the_street":5,
        "number_of_frontages":4
    })),
     ("lowerbound_trimming", LowerBoundTrimmer(column_boundaries={
        "co2_emission":0,
        "cadastral_income":4,
        "garden_surface":0,
        "kitchen_surface":1,
        "living_area":3,
        "street_frontage_width":0,
        "surface_of_the_plot":0,
        "width_of_the_lot_on_the_street": 0,
        "number_of_frontages":1,
        "price": 11
     })), 
    ('filling na', FillNaColumnTransformer(fill_value='Missing', columns=[
        "surroundings_type",
        "available_as_of",
        "building_condition",
        "kitchen_type",
        "energy_class", 
        "heating_type"
    ])),
    ('filling na2', FillNaColumnTransformer(fill_value=0, columns=categorical_features)),     
     ("one_hot_encoding", OneHotColumnEncoder(columns=[
        "surroundings_type",
        "available_as_of",
        "building_condition",
        "number_of_frontages",
        "kitchen_type",
        "heating_type",
    ])),
    ("ordinal_encoding", OrdinalColumnEncoder(columns=[
        "energy_class",
        ])),
    ("KNN_imputing", KNNColumnImputer(n_neighbors=5,columns=numerical_features))
])

In [ ]:

prepared_train_set = data=pipeline.fit_transform(train_set)

In [ ]:
y_columns = ["price"]
X_columns =list(prepared_train_set.columns)
X_columns.remove("price")

In [ ]:
param_distributions = [{
'n_estimators': [1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000],
"max_depth": [1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50,60,70,80,90,100],
"min_samples_split": [1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50],
"min_samples_leaf": [1,2,3,4,5,6,7,8,9,10,15,20,25,30,35,40,45,50],
}]

rnd_search = HalvingRandomSearchCV(RandomForestRegressor(random_state=42), param_distributions, cv=3, scoring='neg_mean_squared_error', verbose=10)
rnd_search.fit(prepared_train_set[X_columns], np.ravel(prepared_train_set[y_columns]))


In [ ]:
rnd_search.best_params_

In [ ]:
cv_res = pd.DataFrame(rnd_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

In [ ]:

prepared_test_set = data=pipeline.fit_transform(test_set)

In [ ]:
from sklearn.metrics import mean_squared_error

for feature in features_to_drop:
    if feature in X_columns:
        X_columns.remove(feature)

final_predictions = rnd_search.best_estimator_.predict(prepared_test_set[X_columns])
final_mse = mean_squared_error(prepared_test_set["price"], final_predictions)
print(final_mse) # prints 41424.40026462184
